In [44]:
from dotenv import load_dotenv
load_dotenv()

True

In [45]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [46]:
import re
import os
from textwrap import dedent
from pprint import pprint
from typing import List

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

In [47]:

# 1. 카페 메뉴 데이터 로드 및 벡터 DB 구축
def create_cafe_vector_db():
    # 카페 메뉴 텍스트 데이터를 로드
    loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
    documents = loader.load()
    
    # 메뉴 항목별로 분할
    def split_menu_items(document):
        pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
        menu_items = re.findall(pattern, document.page_content, re.DOTALL)
        
        menu_documents = []
        for i, item in enumerate(menu_items, 1):
            # 메뉴 이름 추출
            menu_name = item.split('\n')[0].split('.', 1)[1].strip()
            
            menu_doc = Document(
                page_content=item.strip(),
                metadata={
                    "source": document.metadata['source'],
                    "menu_number": i,
                    "menu_name": menu_name
                }
            )
            menu_documents.append(menu_doc)
        
        return menu_documents
    
    # 메뉴 항목 분리 실행
    menu_documents = []
    for doc in documents:
        menu_documents += split_menu_items(doc)
    
    # 임베딩 모델 설정
    embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
    
    # FAISS 인덱스 생성
    cafe_db = FAISS.from_documents(
        documents=menu_documents, 
        embedding=embeddings_model
    )
    
    # FAISS 인덱스 저장
    cafe_db.save_local("./db/cafe_db")
    
    return cafe_db

### 3개 도구 정의 및 LLM에 바인딩

In [48]:

# 1. 카페 메뉴 데이터 로드 및 벡터 DB 구축
def create_cafe_vector_db():
    # 카페 메뉴 텍스트 데이터를 로드
    loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
    documents = loader.load()
    
    # 메뉴 항목별로 분할
    def split_menu_items(document):
        pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
        menu_items = re.findall(pattern, document.page_content, re.DOTALL)
        
        menu_documents = []
        for i, item in enumerate(menu_items, 1):
            # 메뉴 이름 추출
            menu_name = item.split('\n')[0].split('.', 1)[1].strip()
            
            menu_doc = Document(
                page_content=item.strip(),
                metadata={
                    "source": document.metadata['source'],
                    "menu_number": i,
                    "menu_name": menu_name
                }
            )
            menu_documents.append(menu_doc)
        
        return menu_documents
    
    # 메뉴 항목 분리 실행
    menu_documents = []
    for doc in documents:
        menu_documents += split_menu_items(doc)
    
    # 임베딩 모델 설정
    embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
    
    # FAISS 인덱스 생성
    cafe_db = FAISS.from_documents(
        documents=menu_documents, 
        embedding=embeddings_model
    )
    
    # FAISS 인덱스 저장
    cafe_db.save_local("./db/cafe_db")
    
    return cafe_db

In [49]:

# 2. 도구 정의
# 웹 검색 도구
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    
    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    ])
    
    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [50]:

# 위키피디아 검색 도구
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field

def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()
    
    formatted_docs = [
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
    ]
    
    return formatted_docs

class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

In [51]:

# LLM 모델 
#llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm.model_name)

meta-llama/llama-4-scout-17b-16e-instruct


In [52]:

summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm
)

wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

In [53]:

# 카페 메뉴 검색 도구
@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized cafe menu information from the encrypted database.
    Use this tool only for cafe menu-related queries to maintain data confidentiality.
    """
    embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
    cafe_db = FAISS.load_local(
        "./db/cafe_db", 
        embeddings_model, 
        allow_dangerous_deserialization=True
    )
    
    docs = cafe_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 카페 메뉴 정보를 찾을 수 없습니다.")]

In [54]:

# 3. 도구를 LLM에 바인딩
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools=tools)

In [55]:

# 4. 간단한 도구 호출 체인 구현
@chain
def cafe_search_chain(user_input: str, config: RunnableConfig):
    # 첫 번째 LLM 호출로 도구 사용 결정
    ai_msg = llm_with_tools.invoke(user_input, config=config)
    
    # 도구 실행
    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
    
    # 최종 답변 생성을 위한 프롬프트
    final_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful cafe assistant. Provide accurate information based on the search results."),
        ("human", "{user_input}"),
        ("ai", ai_msg.content if ai_msg.content else "도구를 사용하여 정보를 검색했습니다."),
        ("human", "검색 결과: {tool_results}")
    ])
    
    # 도구 결과를 문자열로 변환
    tool_results_str = "\n\n".join([str(msg.content) for msg in tool_msgs])
    
    # 최종 답변 생성
    final_chain = final_prompt | llm
    return final_chain.invoke({
        "user_input": user_input,
        "tool_results": tool_results_str
    }, config=config)

In [56]:

# 5. 실행 및 테스트
if __name__ == "__main__":
    # 벡터 DB 생성 (최초 1회만 실행)
    try:
        create_cafe_vector_db()
        print("카페 메뉴 벡터 DB가 성공적으로 생성되었습니다.")
    except Exception as e:
        print(f"벡터 DB 생성 중 오류: {e}")
    
    # 질문에 답변
    query = "아메리카노의 가격은 얼마인가요? 아메리카노의 유래는 무엇인가?"
    response = cafe_search_chain.invoke(query)
    
    print("질문:", query)
    print("답변:", response.content)               

카페 메뉴 벡터 DB가 성공적으로 생성되었습니다.
db_search_cafe_func: 
{'name': 'db_search_cafe_func', 'args': {'query': '아메리사노 가각'}, 'id': 'q5v7g0gsy', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '아메리사노 유래'}, 'id': 'krj3adbx1', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
질문: 아메리카노의 가격은 얼마인가요? 아메리카노의 유래는 무엇인가?
답변: 아메리카노의 가격은 4,500원입니다.

아메리카노는 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 

아메리카노의 유래는 1940년대 이탈리아에 있습니다. 에스프레소를 즐겨 마시던 이탈리아 사람들이었지만, 미국인들이 에스프레소보다 더 묽은 커피를 선호한다는 사실을 알게 되었습니다. 그래서 에스프레소에 물을 추가한 '아메리카노'가 탄생하게 되었습니다.


## 5-2 Few-shot 프롬프팅

In [58]:
import re
from datetime import datetime
from textwrap import dedent
from pprint import pprint
from typing import List

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field

In [59]:

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized cafe menu information from the encrypted database.
    Use this tool only for cafe menu-related queries to maintain data confidentiality.
    """
    embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
    cafe_db = FAISS.load_local(
        "./db/cafe_db", 
        embeddings_model, 
        allow_dangerous_deserialization=True
    )
    
    docs = cafe_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 카페 메뉴 정보를 찾을 수 없습니다.")]

# 2. Few-shot 예제 정의
examples = [
    HumanMessage("아메리카노의 가격과 특징, 그리고 커피의 역사에 대해 알려주세요.", name="example_user"),
    AIMessage("카페 메뉴를 검색하고, 위키피디아에서 커피 역사 정보를 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}]),
    ToolMessage("아메리카노: 가격 ₩4,500, 에스프레소와 뜨거운 물로 만든 클래식 블랙 커피, 원두 본연의 맛을 느낄 수 있음", tool_call_id="1"),    
    AIMessage("아메리카노 정보를 찾았습니다. 이제 커피의 역사를 위키피디아에서 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "커피 역사"}, "id": "2"}]),
    ToolMessage("커피는 에티오피아에서 기원하여 아랍을 거쳐 전 세계로 전파된 음료입니다. 15세기 예멘에서 처음 재배되기 시작했으며, 17세기 유럽으로 전해져 커피하우스 문화가 발달했습니다. 산업혁명과 함께 대량 생산이 가능해지면서 현재와 같은 커피 문화가 형성되었습니다.", tool_call_id="2"),
    AIMessage("아메리카노(₩4,500)는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 커피는 에티오피아에서 기원하여 아랍을 거쳐 전 세계로 전파되었으며, 15세기 예멘에서 처음 재배되기 시작했습니다. 17세기 유럽으로 전해져 커피하우스 문화가 발달했고, 산업혁명과 함께 대량 생산이 가능해지면서 현재와 같은 커피 문화가 형성되었습니다.", name="example_assistant"),
]

# 3. Few-shot 프롬프트 템플릿 작성
today = datetime.today().strftime("%Y-%m-%d")

system = """You are an AI assistant providing cafe menu information and general food/beverage-related knowledge.
For information about the cafe's menu, use the db_search_cafe_func tool.
For other general information about food, beverages, and their history, use the wiki_summary tool.
If additional web searches are needed or for the most up-to-date information, use the tavily_search_func tool.

Guidelines:
1. For cafe menu queries, always search the menu database first
2. For general knowledge about food/beverages, use Wikipedia
3. For latest trends or news, use web search
4. Provide comprehensive answers by combining multiple sources when needed
5. Always cite the source of your information
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f" Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# 4. 도구를 LLM에 바인딩
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools=tools)

# 5. Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 6. 도구 실행 결과를 종합하여 최종 답변을 생성하는 체인 구현
@chain
def advanced_cafe_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)
    
    print("AI가 선택한 도구들:")
    for tool_call in ai_msg.tool_calls:
        print(f"- {tool_call['name']}: {tool_call['args']}")
    print("-" * 50)
    
    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"실행 중: {tool_call['name']}")
        
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
    
    print(f"총 {len(tool_msgs)}개의 도구 결과를 받았습니다.")
    print("-" * 50)
    
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

In [60]:

# 7. 실행 및 테스트
if __name__ == "__main__":
    # 복합 질문 테스트
    query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
    
    print(f"질문: {query}")
    print("=" * 80)
    
    response = advanced_cafe_chain.invoke(query)
    
    print("최종 답변:")
    print(response.content)
    
    # 추가 테스트 질문들
    additional_queries = [
        "콜드브루의 가격과 특징을 알려주세요.",
        "티라미수와 잘 어울리는 커피는 무엇인가요? 티라미수의 유래도 설명해주세요.",
        "최근 커피 트렌드는 무엇인가요?"
    ]
    
    print("\n" + "=" * 80)
    print("추가 테스트 질문들:")
    
    for i, additional_query in enumerate(additional_queries, 1):
        print(f"\n{i}. {additional_query}")
        try:
            additional_response = advanced_cafe_chain.invoke(additional_query)
            print(f"답변: {additional_response.content[:200]}...")  # 처음 200자만 출력
        except Exception as e:
            print(f"오류 발생: {e}")

질문: 카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요.
AI가 선택한 도구들:
- db_search_cafe_func: {'query': '카페라떼 디저트'}
--------------------------------------------------
실행 중: db_search_cafe_func
총 1개의 도구 결과를 받았습니다.
--------------------------------------------------
최종 답변:
카페라떼는 다양한 디저트와 잘 어울립니다. 특히 크림을 사용한 디저트인 티라미수, 크림 케이크, 마카롱 등이 추천됩니다. 이제 라떼의 유래를 알려드리겠습니다.

추가 테스트 질문들:

1. 콜드브루의 가격과 특징을 알려주세요.
AI가 선택한 도구들:
- db_search_cafe_func: {'query': '콜드브루'}
--------------------------------------------------
실행 중: db_search_cafe_func
총 1개의 도구 결과를 받았습니다.
--------------------------------------------------
답변: 콜드브루의 가격은 ₩5,000이며, 찬물에 12-24시간 우려낸 콜드브루 원액을 사용한 시원한 커피입니다. 부드럽고 달콤한 맛이 특징이며, 산미가 적어 누구나 부담 없이 즐길 수 있습니다. 얼음과 함께 시원하게 제공됩니다....

2. 티라미수와 잘 어울리는 커피는 무엇인가요? 티라미수의 유래도 설명해주세요.
AI가 선택한 도구들:
- wiki_summary: {'query': '티라미수 유래'}
--------------------------------------------------
실행 중: wiki_summary
총 1개의 도구 결과를 받았습니다.
--------------------------------------------------
답변: ...

3. 최근 커피 트렌드는 무엇인가요?
AI가 선택한 

In [61]:

# 8. Gradio UI 구현 (선택사항)
def create_gradio_interface():
    """
    Gradio를 사용한 웹 인터페이스 생성
    """
    import gradio as gr
    from typing import List, Tuple
    
    def chat_interface(message: str, history: List[Tuple[str, str]]) -> str:
        try:
            # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
            chat_history = []
            for human, ai in history[-2:]:  # 최근 2개의 대화만 사용
                chat_history.append(HumanMessage(content=human))
                chat_history.append(AIMessage(content=ai))
            
            # 체인 실행
            response = advanced_cafe_chain.invoke(message)
            return response.content
            
        except Exception as e:
            return f"죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다: {str(e)}"
    
    # 예제 질문들
    example_questions = [
        "아메리카노의 가격과 특징을 알려주세요.",
        "카페라떼와 어울리는 디저트는 무엇인가요?",
        "콜드브루의 유래에 대해 설명해주세요.",
        "최근 커피 트렌드를 검색해주세요."
    ]
    
    # Gradio 인터페이스 생성
    demo = gr.ChatInterface(
        fn=chat_interface,
        title="카페 메뉴 AI 어시스턴트",
        description="카페 메뉴 정보, 음료 추천, 음식 관련 질문에 답변해 드립니다.",
        examples=example_questions,
        theme=gr.themes.Soft()
    )
    
    return demo

# 실행 예시
if __name__ == "__main__":
    # 기본 콘솔 테스트
    print("=== 카페 AI 어시스턴트 테스트 ===")
    
    # 웹 인터페이스 실행 (선택사항)
    demo = create_gradio_interface()
    demo.launch()
    
    print("테스트 완료!")

=== 카페 AI 어시스턴트 테스트 ===
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


테스트 완료!


In [ ]:

# 데모 종료
demo.close()

Closing server running on port: 7860


: 